##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Basic regression: Predict fuel efficiency

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://tensorflow.org/tutorials/keras/regression"><img src="https://tensorflow.org/images/tf_logo_32px.png" />在 tensorflow.org 上查看</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/regression.ipynb"><img src="https://tensorflow.org/images/colab_logo_32px.png" />在 Google Colab 中運行</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/keras/regression.ipynb"><img src="https://tensorflow.org/images/GitHub-Mark-32px.png" />在 GitHub 上查看源代碼</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/keras/regression.ipynb"><img src="https://tensorflow.org/images/download_logo_32px.png" />下載 notebook</a>
  </td>
</table>

Note: 我們的 TensorFlow 社區翻譯了這些文檔。因為社區翻譯是盡力而為， 所以無法保證它們是最準確的，並且反映了最新的
[官方英文文檔](https://www.tensorflow.org/?hl=en)。如果您有改進此翻譯的建議， 請提交 pull request 到
[tensorflow/docs](https://github.com/tensorflow/docs) GitHub 倉庫。要志願地撰寫或者審核譯文，請加入
[docs@tensorflow.org Google Group](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs)。

在 *回歸 (regression)* 問題中，我們的目的是預測出如價格或概率這樣連續值的輸出。相對於*分類(classification)* 問題，*分類(classification)* 的目的是從一系列的分類出選擇出一個分類（如，給出一張包含蘋果或橘子的圖片，識別出圖片中是哪種水果）。

本 notebook 使用經典的 [Auto MPG](https://archive.ics.uci.edu/ml/datasets/auto+mpg) 數據集，構建了一個用來預測70年代末到80年代初汽車燃油效率的模型。為了做到這一點，我們將為該模型提供許多那個時期的汽車描述。這個描述包含：氣缸數，排量，馬力以及重量。

本示例使用 `tf.keras` API，相關細節請參閱 [本指南](https://tensorflow.org/guide/keras)。

In [0]:
# 使用 seaborn 繪製矩陣圖 (pairplot)
!pip install seaborn

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

## Auto MPG 數據集

該數據集可以從 [UCI機器學習庫](https://archive.ics.uci.edu/ml/) 中獲取.



### 獲取數據
首先下載數據集。

In [0]:
dataset_path = keras.utils.get_file("auto-mpg.data", "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")
dataset_path

使用 pandas 導入數據集。

In [0]:
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight',
                'Acceleration', 'Model Year', 'Origin']
raw_dataset = pd.read_csv(dataset_path, names=column_names,
                      na_values = "?", comment='\t',
                      sep=" ", skipinitialspace=True)

dataset = raw_dataset.copy()
dataset.tail()

### 數據清洗

數據集中包括一些未知值。

In [0]:
dataset.isna().sum()

為了保證這個初始示例的簡單性，刪除這些行。

In [0]:
dataset = dataset.dropna()

`"Origin"` 列實際上代表分類，而不僅僅是一個數字。所以把它轉換為獨熱碼 （one-hot）:

In [0]:
origin = dataset.pop('Origin')

In [0]:
dataset['USA'] = (origin == 1)*1.0
dataset['Europe'] = (origin == 2)*1.0
dataset['Japan'] = (origin == 3)*1.0
dataset.tail()

### 拆分訓練數據集和測試數據集

現在需要將數據集拆分為一個訓練數據集和一個測試數據集。

我們最後將使用測試數據集對模型進行評估。

In [0]:
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)

### 數據檢查

快速查看訓練集中幾對列的聯合分佈。

In [0]:
sns.pairplot(train_dataset[["MPG", "Cylinders", "Displacement", "Weight"]], diag_kind="kde")

也可以查看總體的數據統計:

In [0]:
train_stats = train_dataset.describe()
train_stats.pop("MPG")
train_stats = train_stats.transpose()
train_stats

### 從標籤中分離特徵

將特徵值從目標值或者"標籤"中分離。這個標籤是你使用訓練模型進行預測的值。

In [0]:
train_labels = train_dataset.pop('MPG')
test_labels = test_dataset.pop('MPG')

### 數據規範化

再次審視下上面的 `train_stats` 部分，並註意每個特徵的範圍有什麼不同。

使用不同的尺度和範圍對特徵歸一化是好的實踐。儘管模型*可能* 在沒有特徵歸一化的情況下收斂，它會使得模型訓練更加複雜，並會造成生成的模型依賴輸入所使用的單位選擇。

注意：儘管我們僅僅從訓練集中有意生成這些統計數據，但是這些統計信息也會用於歸一化的測試數據集。我們需要這樣做，將測試數據集放入到與已經訓練過的模型相同的分佈中。

In [0]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

我們將會使用這個已經歸一化的數據來訓練模型。

警告: 用於歸一化輸入的數據統計（均值和標準差）需要反饋給模型從而應用於任何其他數據，以及我們之前所獲得獨熱碼。這些數據包含測試數據集以及生產環境中所使用的實時數據。

## 模型

### 構建模型

讓我們來構建我們自己的模型。這裡，我們將會使用一個“順序”模型，其中包含兩個緊密相連的隱藏層，以及返回單個、連續值得輸出層。模型的構建步驟包含於一個名叫 'build_model' 的函數中，稍後我們將會創建第二個模型。兩個密集連接的隱藏層。

In [0]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [0]:
model = build_model()

### 檢查模型

使用 `.summary` 方法來打印該模型的簡單描述。

In [0]:
model.summary()


現在試用下這個模型。從訓練數據中批量獲取‘10’條例子並對這些例子調用 `model.predict` 。


In [0]:
example_batch = normed_train_data[:10]
example_result = model.predict(example_batch)
example_result

它似乎在工作，並產生了預期的形狀和類型的結果

### 訓練模型

對模型進行1000個週期的訓練，並在 `history` 對像中記錄訓練和驗證的準確性。

In [0]:
# 通過為每個完成的時期打印一個點來顯示訓練進度
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 1000

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])

使用 `history` 對像中存儲的統計信息可視化模型的訓練進度。

In [0]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [0]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [MPG]')
  plt.plot(hist['epoch'], hist['mae'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mae'],
           label = 'Val Error')
  plt.ylim([0,5])
  plt.legend()

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [$MPG^2$]')
  plt.plot(hist['epoch'], hist['mse'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mse'],
           label = 'Val Error')
  plt.ylim([0,20])
  plt.legend()
  plt.show()


plot_history(history)

該圖表顯示在約100個 epochs 之後誤差非但沒有改進，反而出現惡化。讓我們更新 `model.fit` 調用，當驗證值沒有提高上是自動停止訓練。
我們將使用一個 *EarlyStopping callback* 來測試每個 epoch 的訓練條件。如果經過一定數量的 epochs 後沒有改進，則自動停止訓練。

你可以從[這裡](https://tensorflow.org/versions/master/api_docs/python/tf/keras/callbacks/EarlyStopping)學習到更多的回調。

In [0]:
model = build_model()

# patience 值用來檢查改進 epochs 的數量
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(normed_train_data, train_labels, epochs=EPOCHS,
                    validation_split = 0.2, verbose=0, callbacks=[early_stop, PrintDot()])

plot_history(history)

如圖所示，驗證集中的平均的誤差通常在 +/- 2 MPG左右。這個結果好麼？我們將決定權留給你。

讓我們看看通過使用 **測試集** 來泛化模型的效果如何，我們在訓練模型時沒有使用測試集。這告訴我們，當我們在現實世界中使用這個模型時，我們可以期望它預測得有多好。

In [0]:
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} MPG".format(mae))

### 做預測
 
最後，使用測試集中的數據預測 MPG 值:

In [0]:
test_predictions = model.predict(normed_test_data).flatten()

plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [MPG]')
plt.ylabel('Predictions [MPG]')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100, 100], [-100, 100])


這看起來我們的模型預測得相當好。我們來看下誤差分佈。

In [0]:
error = test_predictions - test_labels
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error [MPG]")
_ = plt.ylabel("Count")

它不是完全的高斯分佈，但我們可以推斷出，這是因為樣本的數量很小所導致的。

## 结论

本筆記本 (notebook) 介紹了一些處理回歸問題的技術。

* 均方誤差（MSE）是用於回歸問題的常見損失函數（分類問題中使用不同的損失函數）。
* 類似的，用於回歸的評估指標與分類不同。常見的回​​歸指標是平均絕對誤差（MAE）。
* 當數字輸入數據特徵的值存在不同範圍時，每個特徵應獨立縮放到相同範圍。
* 如果訓練數據不多，一種方法是選擇隱藏層較少的小網絡，以避免過度擬合。
* 早期停止是一種防止過度擬合的有效技術。